#### import libraries 

In [1]:
import pandas as pd
import pickle
import numpy as np
from datetime import datetime
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
# from kneed import KneeLocator
# import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('customer.csv')
df 



,customer_id,age,gender,location,membership_start_date,average_purchase_value,purchase_frequency_per_month,last_purchase_date,product_preference,preferred_channel,store_visit_frequency,days_since_last_visit,product_segment,Unnamed: 13,Mobile
0,CUST1000,58,Male,East Jessetown,30-08-2020,375.94,2.6,21-10-2023,Gym Gear,In-Store,17,108,Casual Gym-Goer,NaN,8281359250
1,CUST1001,20,Male,Herrerafurt,21-01-2025,124.95,5.2,18-02-2025,Running Shoes,In-Store,20,107,Casual Gym-Goer,NaN,8281359250
2,CUST1002,32,Female,Shawhaven,06-01-2024,153.53,8.7,13-04-2024,Gym Gear,Online,10,55,General Buyer,NaN,8281359250
3,CUST1003,39,Male,Herrerafurt,17-09-2020,202.37,3.7,02-05-2023,Supplements,Online,1,137,Health Conscious Buyer,NaN,8281359250
4,CUST1004,25,Female,Cassandraton,17-06-2023,284.98,8.3,24-02-2025,Training Wear,Online,18,17,Casual Gym-Goer,NaN,8281359250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,CUST1995,62,Male,North Susan,11-01-2025,173.67,5.1,12-04-2025,Running Shoes,Online,18,55,Casual Gym-Goer,NaN,8281359250
996,CUST1996,49,Female,Lake Lisaport,06-12-2024,483.92,0.8,10-05-2025,Supplements,In-Store,6,57,Fitness Enthusiast,NaN,8281359250
997,CUST1997,62,Other,Lisatown,20-03-2022,463.12,7.8,08-04-2023,Yoga Accessories,Online,9,168,General Buyer,NaN,8281359250
998,CUST1998,28,Male,Saramouth,20-01-2021,208.30,4.3,07-11-2024,Yoga Accessories,Online,12,135,General Buyer,NaN,8281359250


In [3]:
df['membership_start_date'] = pd.to_datetime(df['membership_start_date'], dayfirst=True)
df['last_purchase_date'] = pd.to_datetime(df['last_purchase_date'], dayfirst=True)
df['product_preference'] = df['product_preference'].str.lower().str.strip()
df['product_segment'] = df['product_segment'].str.lower().str.strip() 
df

,customer_id,age,gender,location,membership_start_date,average_purchase_value,purchase_frequency_per_month,last_purchase_date,product_preference,preferred_channel,store_visit_frequency,days_since_last_visit,product_segment,Unnamed: 13,Mobile
0,CUST1000,58,Male,East Jessetown,2020-08-30,375.94,2.6,2023-10-21,gym gear,In-Store,17,108,casual gym-goer,NaN,8281359250
1,CUST1001,20,Male,Herrerafurt,2025-01-21,124.95,5.2,2025-02-18,running shoes,In-Store,20,107,casual gym-goer,NaN,8281359250
2,CUST1002,32,Female,Shawhaven,2024-01-06,153.53,8.7,2024-04-13,gym gear,Online,10,55,general buyer,NaN,8281359250
3,CUST1003,39,Male,Herrerafurt,2020-09-17,202.37,3.7,2023-05-02,supplements,Online,1,137,health conscious buyer,NaN,8281359250
4,CUST1004,25,Female,Cassandraton,2023-06-17,284.98,8.3,2025-02-24,training wear,Online,18,17,casual gym-goer,NaN,8281359250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,CUST1995,62,Male,North Susan,2025-01-11,173.67,5.1,2025-04-12,running shoes,Online,18,55,casual gym-goer,NaN,8281359250
996,CUST1996,49,Female,Lake Lisaport,2024-12-06,483.92,0.8,2025-05-10,supplements,In-Store,6,57,fitness enthusiast,NaN,8281359250
997,CUST1997,62,Other,Lisatown,2022-03-20,463.12,7.8,2023-04-08,yoga accessories,Online,9,168,general buyer,NaN,8281359250
998,CUST1998,28,Male,Saramouth,2021-01-20,208.30,4.3,2024-11-07,yoga accessories,Online,12,135,general buyer,NaN,8281359250


In [4]:
d1=df.copy()
d1

,customer_id,age,gender,location,membership_start_date,average_purchase_value,purchase_frequency_per_month,last_purchase_date,product_preference,preferred_channel,store_visit_frequency,days_since_last_visit,product_segment,Unnamed: 13,Mobile
0,CUST1000,58,Male,East Jessetown,2020-08-30,375.94,2.6,2023-10-21,gym gear,In-Store,17,108,casual gym-goer,NaN,8281359250
1,CUST1001,20,Male,Herrerafurt,2025-01-21,124.95,5.2,2025-02-18,running shoes,In-Store,20,107,casual gym-goer,NaN,8281359250
2,CUST1002,32,Female,Shawhaven,2024-01-06,153.53,8.7,2024-04-13,gym gear,Online,10,55,general buyer,NaN,8281359250
3,CUST1003,39,Male,Herrerafurt,2020-09-17,202.37,3.7,2023-05-02,supplements,Online,1,137,health conscious buyer,NaN,8281359250
4,CUST1004,25,Female,Cassandraton,2023-06-17,284.98,8.3,2025-02-24,training wear,Online,18,17,casual gym-goer,NaN,8281359250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,CUST1995,62,Male,North Susan,2025-01-11,173.67,5.1,2025-04-12,running shoes,Online,18,55,casual gym-goer,NaN,8281359250
996,CUST1996,49,Female,Lake Lisaport,2024-12-06,483.92,0.8,2025-05-10,supplements,In-Store,6,57,fitness enthusiast,NaN,8281359250
997,CUST1997,62,Other,Lisatown,2022-03-20,463.12,7.8,2023-04-08,yoga accessories,Online,9,168,general buyer,NaN,8281359250
998,CUST1998,28,Male,Saramouth,2021-01-20,208.30,4.3,2024-11-07,yoga accessories,Online,12,135,general buyer,NaN,8281359250


### Feature engineering 

In [5]:
reference_date = d1['last_purchase_date'].max()

d1['Active_days'] = ((reference_date - d1['membership_start_date']).dt.days).round().astype(int)
# d1['Avg_purchase_gap_days'] = (d1['Active_days'] /d1['store_visit_frequency'])
d1['Avg_purchase_gap_days'] = d1.apply(lambda x: x['Active_days'] / x['store_visit_frequency'] if x['store_visit_frequency'] > 0 else x['Active_days'], axis=1)
d1['Frequency'] = (d1['purchase_frequency_per_month'] * ((d1['last_purchase_date'] - d1['membership_start_date']).dt.days / 30.0)).round().astype(int)
d1['Frequency']=d1['Frequency'].astype(int)
d1['Monetary'] = d1['Frequency'] * d1['average_purchase_value']
d1['Recency'] = (datetime.today() - d1['last_purchase_date']).dt.days  


d1

,customer_id,age,gender,location,membership_start_date,average_purchase_value,purchase_frequency_per_month,last_purchase_date,product_preference,preferred_channel,store_visit_frequency,days_since_last_visit,product_segment,Unnamed: 13,Mobile,Active_days,Avg_purchase_gap_days,Frequency,Monetary,Recency
0,CUST1000,58,Male,East Jessetown,2020-08-30,375.94,2.6,2023-10-21,gym gear,In-Store,17,108,casual gym-goer,NaN,8281359250,1715,100.882353,99,37218.06,579
1,CUST1001,20,Male,Herrerafurt,2025-01-21,124.95,5.2,2025-02-18,running shoes,In-Store,20,107,casual gym-goer,NaN,8281359250,110,5.500000,5,624.75,93
2,CUST1002,32,Female,Shawhaven,2024-01-06,153.53,8.7,2024-04-13,gym gear,Online,10,55,general buyer,NaN,8281359250,491,49.100000,28,4298.84,404
3,CUST1003,39,Male,Herrerafurt,2020-09-17,202.37,3.7,2023-05-02,supplements,Online,1,137,health conscious buyer,NaN,8281359250,1697,1697.000000,118,23879.66,751
4,CUST1004,25,Female,Cassandraton,2023-06-17,284.98,8.3,2025-02-24,training wear,Online,18,17,casual gym-goer,NaN,8281359250,694,38.555556,171,48731.58,87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,CUST1995,62,Male,North Susan,2025-01-11,173.67,5.1,2025-04-12,running shoes,Online,18,55,casual gym-goer,NaN,8281359250,120,6.666667,15,2605.05,40
996,CUST1996,49,Female,Lake Lisaport,2024-12-06,483.92,0.8,2025-05-10,supplements,In-Store,6,57,fitness enthusiast,NaN,8281359250,156,26.000000,4,1935.68,12
997,CUST1997,62,Other,Lisatown,2022-03-20,463.12,7.8,2023-04-08,yoga accessories,Online,9,168,general buyer,NaN,8281359250,1148,127.555556,100,46312.00,775
998,CUST1998,28,Male,Saramouth,2021-01-20,208.30,4.3,2024-11-07,yoga accessories,Online,12,135,general buyer,NaN,8281359250,1572,131.000000,199,41451.70,196


In [6]:
d1.rename(columns={'average_purchase_value': 'Avg_monetary','product_segment':'Product_category'}, inplace=True)
data=d1[['product_preference','store_visit_frequency','Active_days','days_since_last_visit','Avg_purchase_gap_days','Recency','Monetary','Frequency','Avg_monetary',]]  
cleaned_data=data    
data

,product_preference,store_visit_frequency,Active_days,days_since_last_visit,Avg_purchase_gap_days,Recency,Monetary,Frequency,Avg_monetary
0,gym gear,17,1715,108,100.882353,579,37218.06,99,375.94
1,running shoes,20,110,107,5.500000,93,624.75,5,124.95
2,gym gear,10,491,55,49.100000,404,4298.84,28,153.53
3,supplements,1,1697,137,1697.000000,751,23879.66,118,202.37
4,training wear,18,694,17,38.555556,87,48731.58,171,284.98
...,...,...,...,...,...,...,...,...,...
995,running shoes,18,120,55,6.666667,40,2605.05,15,173.67
996,supplements,6,156,57,26.000000,12,1935.68,4,483.92
997,yoga accessories,9,1148,168,127.555556,775,46312.00,100,463.12
998,yoga accessories,12,1572,135,131.000000,196,41451.70,199,208.30


In [7]:
product_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
product_onehotencoder = product_encoder.fit_transform(data[['product_preference']])
product_onehotencoder

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.]])

In [8]:
product_encoder.get_feature_names_out(['product_preference'])

array(['product_preference_gym gear', 'product_preference_running shoes',
       'product_preference_supplements',
       'product_preference_training wear',
       'product_preference_yoga accessories'], dtype=object)

In [9]:
Encoder_data = pd.DataFrame(product_onehotencoder, columns=product_encoder.get_feature_names_out(['product_preference']))
Encoder_data

,product_preference_gym gear,product_preference_running shoes,product_preference_supplements,product_preference_training wear,product_preference_yoga accessories
0,1.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...
995,0.0,1.0,0.0,0.0,0.0
996,0.0,0.0,1.0,0.0,0.0
997,0.0,0.0,0.0,0.0,1.0
998,0.0,0.0,0.0,0.0,1.0


In [10]:
data=pd.concat([data.drop('product_preference',axis=1),Encoder_data],axis=1)
data

,store_visit_frequency,Active_days,days_since_last_visit,Avg_purchase_gap_days,Recency,Monetary,Frequency,Avg_monetary,product_preference_gym gear,product_preference_running shoes,product_preference_supplements,product_preference_training wear,product_preference_yoga accessories
0,17,1715,108,100.882353,579,37218.06,99,375.94,1.0,0.0,0.0,0.0,0.0
1,20,110,107,5.500000,93,624.75,5,124.95,0.0,1.0,0.0,0.0,0.0
2,10,491,55,49.100000,404,4298.84,28,153.53,1.0,0.0,0.0,0.0,0.0
3,1,1697,137,1697.000000,751,23879.66,118,202.37,0.0,0.0,1.0,0.0,0.0
4,18,694,17,38.555556,87,48731.58,171,284.98,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,18,120,55,6.666667,40,2605.05,15,173.67,0.0,1.0,0.0,0.0,0.0
996,6,156,57,26.000000,12,1935.68,4,483.92,0.0,0.0,1.0,0.0,0.0
997,9,1148,168,127.555556,775,46312.00,100,463.12,0.0,0.0,0.0,0.0,1.0
998,12,1572,135,131.000000,196,41451.70,199,208.30,0.0,0.0,0.0,0.0,1.0


In [11]:
with open('Models/onehot_encoder_product_prefreence.pkl','wb') as file:
    pickle.dump(product_encoder,file)

In [12]:
data

,store_visit_frequency,Active_days,days_since_last_visit,Avg_purchase_gap_days,Recency,Monetary,Frequency,Avg_monetary,product_preference_gym gear,product_preference_running shoes,product_preference_supplements,product_preference_training wear,product_preference_yoga accessories
0,17,1715,108,100.882353,579,37218.06,99,375.94,1.0,0.0,0.0,0.0,0.0
1,20,110,107,5.500000,93,624.75,5,124.95,0.0,1.0,0.0,0.0,0.0
2,10,491,55,49.100000,404,4298.84,28,153.53,1.0,0.0,0.0,0.0,0.0
3,1,1697,137,1697.000000,751,23879.66,118,202.37,0.0,0.0,1.0,0.0,0.0
4,18,694,17,38.555556,87,48731.58,171,284.98,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,18,120,55,6.666667,40,2605.05,15,173.67,0.0,1.0,0.0,0.0,0.0
996,6,156,57,26.000000,12,1935.68,4,483.92,0.0,0.0,1.0,0.0,0.0
997,9,1148,168,127.555556,775,46312.00,100,463.12,0.0,0.0,0.0,0.0,1.0
998,12,1572,135,131.000000,196,41451.70,199,208.30,0.0,0.0,0.0,0.0,1.0


In [13]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)
scaled_data

array([[ 1.08922383,  1.54336224,  0.30980613, ..., -0.51868635,
        -0.52644013, -0.5       ],
       [ 1.592407  , -1.56698147,  0.29110367, ..., -0.51868635,
        -0.52644013, -0.5       ],
       [-0.08487023, -0.8286382 , -0.68142386, ..., -0.51868635,
        -0.52644013, -0.5       ],
       ...,
       [-0.25259795,  0.44456792,  1.43195327, ..., -0.51868635,
        -0.52644013,  2.        ],
       [ 0.25058522,  1.26624127,  0.81477234, ..., -0.51868635,
        -0.52644013,  2.        ],
       [-1.25896429, -1.4197004 ,  1.05790422, ..., -0.51868635,
         1.89955124, -0.5       ]])

In [14]:
with open('Models/scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

#### Features

In [15]:
# features = d1[['product_preference', 'store_visit_frequency', 'Active_days', 'Purchase_count',
#                'Avg_purchase_gap_days', 'monetary', 'Avg_monetary', 'Recency']]


# categorical_features = ['product_preference']
# numerical_features = ['store_visit_frequency', 'Active_days', 'Purchase_count',
#                       'Avg_purchase_gap_days', 'monetary', 'Avg_monetary', 'Recency']

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('cat', OneHotEncoder(sparse_output=False, drop='first'), categorical_features),
#         ('num', StandardScaler(), numerical_features)
#     ]
# )

# # Step 4: Apply transformation
# processed = preprocessor.fit_transform(features)

In [16]:
# inertia = []
# K_range = range(1, 11)

# for k in K_range:
#     kmeans = KMeans(n_clusters=k, random_state=42)
#     kmeans.fit(scaled_data)
#     inertia.append(kmeans.inertia_)

# knee = KneeLocator(K_range, inertia, curve='convex', direction='decreasing')
# optimal_k = knee.knee

# print(f"Optimal number of clusters: {optimal_k}")


In [17]:
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans.fit(scaled_data)

cleaned_data['cluster'] = kmeans.labels_
cleaned_data

,product_preference,store_visit_frequency,Active_days,days_since_last_visit,Avg_purchase_gap_days,Recency,Monetary,Frequency,Avg_monetary,cluster
0,gym gear,17,1715,108,100.882353,579,37218.06,99,375.94,1
1,running shoes,20,110,107,5.500000,93,624.75,5,124.95,0
2,gym gear,10,491,55,49.100000,404,4298.84,28,153.53,1
3,supplements,1,1697,137,1697.000000,751,23879.66,118,202.37,3
4,training wear,18,694,17,38.555556,87,48731.58,171,284.98,1
...,...,...,...,...,...,...,...,...,...,...
995,running shoes,18,120,55,6.666667,40,2605.05,15,173.67,0
996,supplements,6,156,57,26.000000,12,1935.68,4,483.92,3
997,yoga accessories,9,1148,168,127.555556,775,46312.00,100,463.12,2
998,yoga accessories,12,1572,135,131.000000,196,41451.70,199,208.30,2


In [18]:
with open('Models/kmeans_model.pkl', 'wb') as file:
    pickle.dump(kmeans, file)

In [19]:
# grouped = cleaned_data.groupby('cluster')['Monetary'].sum().sort_values(ascending=False).index
# grouped

In [20]:
# print(grouped)
# tier_names = ['Platinum', 'Gold', 'Silver', 'Bronze']
# loyalty_tier_map = {cluster_id: tier for cluster_id, tier in zip(grouped, tier_names)}
# print(loyalty_tier_map)
# cleaned_data['loyalty_tier'] = cleaned_data['cluster'].map(loyalty_tier_map)
# cleaned_data

In [21]:
agg = cleaned_data.groupby(['cluster']).agg({
    'Frequency': 'sum',
    'Monetary': 'sum'
}).rename(columns={
    'Frequency': 'Frequency',
    'Monetary': 'Monetary'
}).reset_index()

agg['Unit Price'] = (agg['Monetary'] / agg['Frequency'])
agg



,cluster,Frequency,Monetary,Unit Price
0,0,13383,3354460.77,250.650883
1,1,32834,8578452.29,261.267354
2,2,16579,4255496.14,256.679905
3,3,18592,5008885.44,269.410792


In [22]:
tier_labels = ['Platinum', 'Gold', 'Silver', 'Bronze']

agg = agg.sort_values('Unit Price', ascending=False)
agg['loyalty'] = tier_labels[:len(agg)]
agg

,cluster,Frequency,Monetary,Unit Price,loyalty
3,3,18592,5008885.44,269.410792,Platinum
1,1,32834,8578452.29,261.267354,Gold
2,2,16579,4255496.14,256.679905,Silver
0,0,13383,3354460.77,250.650883,Bronze


In [23]:
reward_mapping = {
    'Platinum': '25% discount + VIP concierge access',
    'Gold': '20% discount + free shipping',
    'Silver': '15% discount or birthday bonus',
    'Bronze': 'Points-based rewards or 10% discount'}



In [24]:
agg['assigned_reward'] = agg['loyalty'].map(reward_mapping)
agg=agg[['cluster','loyalty','assigned_reward']]

In [25]:
final = cleaned_data.merge(agg, on='cluster', how='left')
final

,product_preference,store_visit_frequency,Active_days,days_since_last_visit,Avg_purchase_gap_days,Recency,Monetary,Frequency,Avg_monetary,cluster,loyalty,assigned_reward
0,gym gear,17,1715,108,100.882353,579,37218.06,99,375.94,1,Gold,20% discount + free shipping
1,running shoes,20,110,107,5.500000,93,624.75,5,124.95,0,Bronze,Points-based rewards or 10% discount
2,gym gear,10,491,55,49.100000,404,4298.84,28,153.53,1,Gold,20% discount + free shipping
3,supplements,1,1697,137,1697.000000,751,23879.66,118,202.37,3,Platinum,25% discount + VIP concierge access
4,training wear,18,694,17,38.555556,87,48731.58,171,284.98,1,Gold,20% discount + free shipping
...,...,...,...,...,...,...,...,...,...,...,...,...
995,running shoes,18,120,55,6.666667,40,2605.05,15,173.67,0,Bronze,Points-based rewards or 10% discount
996,supplements,6,156,57,26.000000,12,1935.68,4,483.92,3,Platinum,25% discount + VIP concierge access
997,yoga accessories,9,1148,168,127.555556,775,46312.00,100,463.12,2,Silver,15% discount or birthday bonus
998,yoga accessories,12,1572,135,131.000000,196,41451.70,199,208.30,2,Silver,15% discount or birthday bonus


In [26]:
FREQ_THRESHOLD = 5
MONETARY_THRESHOLD = 5000

def apply_reward_rules(row):

    if row['Frequency'] >= FREQ_THRESHOLD or row['Monetary'] >= MONETARY_THRESHOLD:
        return pd.Series([row['loyalty'], row['assigned_reward'], "✅ Eligible for reward"])
    else:
        purchase_gap = max(0, FREQ_THRESHOLD - row['Frequency'])
        money_gap = max(0, MONETARY_THRESHOLD - row['Monetary'])
        msg = f"🔔 You need {purchase_gap} more purchases or ₹{money_gap:.0f} more to earn a reward."
        return pd.Series(['No reward', 'No reward', msg])

final[['loyalty', 'assigned_reward', 'progress_message']] = final.apply(apply_reward_rules, axis=1)

# final_output = final[[
#     'product_preference', 'Frequency', 'Monetary', 'loyalty', 
#     'assigned_reward', 'progress_message'
# ]]

# final_output
final

,product_preference,store_visit_frequency,Active_days,days_since_last_visit,Avg_purchase_gap_days,Recency,Monetary,Frequency,Avg_monetary,cluster,loyalty,assigned_reward,progress_message
0,gym gear,17,1715,108,100.882353,579,37218.06,99,375.94,1,Gold,20% discount + free shipping,✅ Eligible for reward
1,running shoes,20,110,107,5.500000,93,624.75,5,124.95,0,Bronze,Points-based rewards or 10% discount,✅ Eligible for reward
2,gym gear,10,491,55,49.100000,404,4298.84,28,153.53,1,Gold,20% discount + free shipping,✅ Eligible for reward
3,supplements,1,1697,137,1697.000000,751,23879.66,118,202.37,3,Platinum,25% discount + VIP concierge access,✅ Eligible for reward
4,training wear,18,694,17,38.555556,87,48731.58,171,284.98,1,Gold,20% discount + free shipping,✅ Eligible for reward
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,running shoes,18,120,55,6.666667,40,2605.05,15,173.67,0,Bronze,Points-based rewards or 10% discount,✅ Eligible for reward
996,supplements,6,156,57,26.000000,12,1935.68,4,483.92,3,No reward,No reward,🔔 You need 1 more purchases or ₹3064 more to e...
997,yoga accessories,9,1148,168,127.555556,775,46312.00,100,463.12,2,Silver,15% discount or birthday bonus,✅ Eligible for reward
998,yoga accessories,12,1572,135,131.000000,196,41451.70,199,208.30,2,Silver,15% discount or birthday bonus,✅ Eligible for reward


In [27]:
# import pandas as pd

# FREQ_THRESHOLD = 5
# MONETARY_THRESHOLD = 5000

# def assign_rewards_to_new_data(new_data, kmeans_model, scaler, agg):

#     # 1. Scale the new data features (use same scaler as training)
#     features = ['Frequency', 'Monetary', 'Avg_monetary', 'store_visit_frequency', 'Active_days']  # Adjust based on your features
#     scaled_features = scaler.transform(new_data[features])

#     # 2. Predict cluster for new customers
#     new_data['cluster'] = kmeans_model.predict(scaled_features)

#     # 3. Map cluster to loyalty tier and assigned reward
#     new_data = new_data.merge(agg, on='cluster', how='left')

#     # 4. Apply reward eligibility rules
#     def apply_reward_rules(row):
#         if row['Frequency'] >= FREQ_THRESHOLD or row['Monetary'] >= MONETARY_THRESHOLD:
#             return pd.Series([row['loyalty'], row['assigned_reward'], "✅ Eligible for reward"])
#         else:
#             purchase_gap = max(0, FREQ_THRESHOLD - row['Frequency'])
#             money_gap = max(0, MONETARY_THRESHOLD - row['Monetary'])
#             msg = f"🔔 You need {purchase_gap} more purchases or ₹{money_gap:.0f} more to earn a reward."
#             return pd.Series([None, None, msg])

#     new_data[['loyalty', 'assigned_reward', 'progress_message']] = new_data.apply(apply_reward_rules, axis=1)

#     return new_data

